## Populate the Job Posting Ontology

In [2]:
# required libraries
import pandas as pd
import os
from pathlib import Path
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD
# CHECK DATE 
import datetime



In [ ]:
# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
jobPostingUrl = 'data/job_postings.csv'

companiesUrl = path + 'data/company_details/companies.csv'
company_industriesUrl = path + 'data/company_details/company_industries.csv'
company_specialitiesUrl = path + 'data/company_details/company_specialities.csv'
employee_countsUrl = path + 'data/company_details/employee_counts.csv'

benefitsUrl = path + 'data/job_details/benefits.csv'
job_industriesUrl = path + 'data/job_details/job_industries.csv'
job_skillsUrl = path + 'data/job_details/job_skills.csv'
salariesUrl = path + 'data/job_details/salaries.csv'

industriesUrl = path + 'data/maps/industries.csv'
skillsUrl = path + 'data/maps/skills.csv'


# country codes
countriesURL = path + 'data\countries\all.csv'

# saving folder
savePath =  path + '/data/rdf/linkedinDB/' 

In [8]:
# Construct the country and the movie ontology namespaces not known by RDFlib
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
LNJP = Namespace("http://www.dei.unipd.it/database2/LinkedinJobPosting#")
SKOS = Namespace("https://www.w3.org/2009/08/skos-reference/skos-owl1-dl.rdf")



## Job Posting

In [9]:
# Load the CSV files in memory
jobPosting = pd.read_csv(jobPostingUrl, sep=',', index_col='job_id')
#print(jobPosting)

In [18]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)


http://www.dei.unipd.it/database2/LinkedinJobPosting#columns


In [21]:
# Iterate over the job postings DataFrame
for index, row in jobPosting.iterrows():
    # Create the JobPosting node with a URI
    jobPostingId = "jobPosting" + str(index)
    JobPosting = URIRef(LNJP[jobPostingId])
    g.add((JobPosting, RDF.type, LNJP.JobPosting))

    # Add data properties
    g.add((JobPosting, LNJP.sponsored, Literal(row['sponsored'], datatype=XSD.boolean)))
    g.add((JobPosting, LNJP.scraped, Literal(row['scraped'], datatype=XSD.boolean)))
    g.add((JobPosting, LNJP.original_listed_time, Literal(row['original_listed_time'], datatype=XSD.dateTime)))
    g.add((JobPosting, LNJP.applies, Literal(row['applies'], datatype=XSD.integer)))
    g.add((JobPosting, LNJP.expiry, Literal(row['expiry'], datatype=XSD.dateTime)))
    g.add((JobPosting, LNJP.min_salary, Literal(row['min_salary'], datatype=XSD.integer)))
    g.add((JobPosting, LNJP.max_salary, Literal(row['max_salary'], datatype=XSD.integer)))
    g.add((JobPosting, LNJP.location, Literal(row['location'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.work_type, Literal(row['work_type'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.posting_domain, Literal(row['posting_domain'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.compensation_type, Literal(row['compensation_type'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.med_salary, Literal(row['med_salary'], datatype=XSD.integer)))
    g.add((JobPosting, LNJP.formatted_work_type, Literal(row['formatted_work_type'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.skills_desc, Literal(row['skills_desc'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.remote_allowed, Literal(row['remote_allowed'], datatype=XSD.boolean)))
    g.add((JobPosting, LNJP.application_url, Literal(row['application_url'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.pay_period, Literal(row['pay_period'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.listed_time, Literal(row['listed_time'], datatype=XSD.dateTime)))
    g.add((JobPosting, LNJP.currency, Literal(row['currency'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.views, Literal(row['views'], datatype=XSD.integer)))
    g.add((JobPosting, LNJP.company_id, Literal(row['company_id'], datatype=XSD.string)))
    #g.add((JobPosting, LNJP.job_id, Literal(row['job_id'], datatype=XSD.int)))
    g.add((JobPosting, LNJP.job_id, Literal(index, datatype=XSD.int)))
    g.add((JobPosting, LNJP.application_type, Literal(row['application_type'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.job_posting_url, Literal(row['job_posting_url'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.title, Literal(row['title'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.formatted_experience_level, Literal(row['formatted_experience_level'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.closed_time, Literal(row['closed_time'], datatype=XSD.dateTime)))
    g.add((JobPosting, LNJP.description, Literal(row['description'], datatype=XSD.string)))

# Serialize the RDF graph to Turtle format
turtle_file_path = 'job_postings.ttl'
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")

RDF data exported to job_postings.ttl


In [22]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open('soccerDB/leagues.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
CPU times: total: 19.5 s
Wall time: 41.9 s


## Clubs

In [23]:
# Load the CSV files in memory
clubs = pd.read_csv(clubsUrl, sep=',', index_col='club_id')
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("so", SO)

NameError: name 'clubsUrl' is not defined

In [ ]:
%%time 
# Measure execution time

# Iterate over the club dataframe
for index, row in clubs.iterrows():
    # Create the node to add to the Graph
    # The node has the namespace + the club id as URI
    idU = "club"+str(index)
    Club = URIRef(SO[idU])

    # Add triples using store's add() method.
    # Triple: Club rdf:type so:Club .
    g.add((Club, RDF.type, SO.Club))

    # Triple: Club so:name "value_of_row_name"^^xsd:string .
    g.add((Club, SO['name'], Literal(row['name'], datatype=XSD.string)))

    # Triple: Club so:competeIn league_uri .
    idL = "league"+str(row['league_id'])
    g.add((Club, SO['competeIn'], URIRef(SO[idL])))
    
    try:
        # Get the nationality of the club
        nationality = leagues.loc[row['league_id'], 'nationality' ]
        # Create the RDF node
        Country = URIRef(CNS[row['nationality']])
        # Triple: Club so:nationality country_uri .
        g.add((Club, SO['nationality'], Country))
    except KeyError:
        continue

    
    

CPU times: user 19.5 ms, sys: 3.87 ms, total: 23.4 ms
Wall time: 22.8 ms


In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open('soccerDB/clubs.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: user 22.1 ms, sys: 2.59 ms, total: 24.7 ms
Wall time: 23.7 ms


## Players

In [ ]:
# Load the CSV files in memory
# Load players data
players = pd.read_csv(playersUrl, sep=',', index_col='player_id', keep_default_na=False, na_values=['_'])
# Load playersFifa data
playersFifa = pd.read_csv(playersFifaUrl, sep=',', index_col='sofifa_id', keep_default_na=False, na_values=['_'])

# Create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("so", SO)

# Turtle comments for the code above
# Players data:
# :player1 rdf:type so:Player .
# :player1 so:hasAttribute value1 .
# :player1 so:hasAttribute value2 .
# ...

# PlayersFifa data:
# :fifaPlayer1 rdf:type so:FifaPlayer .
# :fifaPlayer1 so:hasFifaAttribute value1 .
# :fifaPlayer1 so:hasFifaAttribute value2 .
# ...

# Note: Replace :player1, :fifaPlayer1, etc., with actual URIs or blank node identifiers.
# Replace so:hasAttribute and so:hasFifaAttribute with actual properties.
# Replace value1, value2, etc., with actual values.


In [ ]:
#load the country codes
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
countries = pd.read_csv(countriesURL, sep=',', index_col='name', keep_default_na=False, na_values=['_'])


In [ ]:
from difflib import SequenceMatcher
import numpy as np

import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

import re

In [ ]:
%%time 
#measure execution time

#iterate over the players dataframe
for index, row in players.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the player id as URI
    idU = "player"+str(index)
    Player = URIRef(SO[idU])
    # the transferMarkt profile has as URI, the URL of the profile in the website
    TransfermarktProfile = URIRef(row['url'])
    
    # Add triples using store's add() method.
    g.add((Player, RDF.type, SO.Player))
    g.add((TransfermarktProfile, RDF.type, SO.TransfermarktProfile))
    g.add((TransfermarktProfile, SO['isAbout'], Player))
    
    #process player name
    name = row['name'].split('-')

    if (len(name)>1):
        g.add((Player, SO['firstName'], Literal(name[0], datatype=XSD.string)))
        g.add((Player, SO['lastName'], Literal(name[1], datatype=XSD.string)))
    else:
        g.add((Player, SO['lastName'], Literal(name[0], datatype=XSD.string)))
        
    #there can be more than one position per player
    for pos in row['position'].split(' - '):
        g.add((Player, SO['position'], Literal(pos.lower(), datatype=XSD.string)))
    
    if not(row['club_id']==''):
        idC = "club"+str(row['club_id'])
        g.add((Player, SO['playFor'], URIRef(SO[idC])))

#iterate over the fifa dataframe
for index, row in playersFifa.iterrows():
    pname = row['short_name'].lower()
    if ('.' in pname):
        # get last name
        # in the fifa dataset we have short names as L. Messi so we delete the L. 
        # we need to check if the last name contains a space
        pname = row['short_name'].split('.')[1].lower().strip()
        if ' ' in pname:
            i = 0
            for t in pname.split(' '):
                if i == 0:
                    pname = t.lower()
                else:
                    pname = pname + "-" + t.lower()
                i += 1           
    elif(' ' in pname):
        # here we have to handle Cristiano Ronaldo mapping it to cristiano-ronaldo to maximize the match in the players dataframe 
        i = 0
        for t in row['short_name'].split(' '):
            if i == 0:
                pname = t.lower()
            else:
                pname = pname + "-" + t.lower()
            i += 1
    pname = strip_accents(pname)
    
    # find sim with the full name 
    fullname = row['long_name'].lower()
    i = 0
    for t in fullname.split(' '):
        if i == 0:
            fullname = t.lower()
        else:
            fullname = fullname + "-" + t.lower()
        i += 1 
    fullname = strip_accents(fullname)
    # check the players with that last name
    names =  players[players['name'].str.contains(pname)]['name']
    #find max similarity    
    maxN = 0
    playerId = ''
    for n in names:
        sim = SequenceMatcher(None, fullname, n).ratio()
        if (maxN < sim):
            maxN = sim
            playerId = players.loc[players['name'] == n].index[0]
        
    #if we get a valid playerId we can connect the Fifa stats to the transfermrkt player
    if (playerId != ''):
        #remove the row from the player dataframe to avoid futher matchings (we know data will contain errors)
        players = players.drop(index=playerId)
        idU = "player"+str(playerId)
        Player = URIRef(SO[idU])
        g.add((Player, SO['overallFifaValue'], Literal(row['overall'], datatype=XSD.int)))
        g.add((Player, SO['growthFifaPotential'], Literal(row['potential'], datatype=XSD.int)))
        g.add((Player, SO['economicValue'], Literal(row['value_eur'], datatype=XSD.int)))
        g.add((Player, SO['annualWage'], Literal(row['wage_eur'], datatype=XSD.int))) 
        
        pFeatures = str(row['player_tags'])
        if pFeatures != '_' and pFeatures != '':
            pFeatures = pFeatures.split(',')
            for feature in pFeatures:
                feature = feature.strip()
                feature = re.sub('#', '', feature)
                g.add((Player, SO['playerFeature'], Literal(feature, datatype=XSD.string)))
        
        if row['contract_valid_until'] != '_' and row['contract_valid_until'] != '':
            g.add((Player, SO['contractValidTo'], Literal(int(row['contract_valid_until']), datatype=XSD.gYear)))        

        g.add((Player, SO['birthday'], Literal(row['dob'], datatype=XSD.date)))
        g.add((Player, SO['height'], Literal(row['height_cm'], datatype=XSD.int)))
        g.add((Player, SO['weight'], Literal(row['weight_kg'], datatype=XSD.int)))
        
        
        nationality = row['nationality'] 
        nationality = nationality.replace(" ", "_")
        # create the RDF node
        Country = URIRef(CNS[nationality])
        # add the edge connecting the Movie and the Country 
        g.add((Player, SO['nationality'], Country))   

        # Homework: extend the code to populate the 'propertyOf' edge
        

CPU times: user 15.5 s, sys: 86.3 ms, total: 15.5 s
Wall time: 15.7 s


In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'players.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: user 1.51 s, sys: 8.25 ms, total: 1.52 s
Wall time: 1.52 s


## Jobs

In [ ]:
# Load the CSV files in memory
apps = pd.read_csv(appearancesUrl, sep=',', index_col='appearance_id', keep_default_na=False, na_values=['_'])
games = pd.read_csv(gamesUrl, sep=',', index_col='game_id', keep_default_na=False, na_values=['_'])

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("so", SO)

In [ ]:
#iterate over the games dataframe
for index, row in games.iterrows():
    # we use the transfermrket URL as URI
    Game = URIRef(row['url'])
    g.add((Game, RDF.type, SO.Game))
    idU1 = "club"+str(row['home_club_id'])
    idU2 = "club"+str(row['away_club_id'])
    HomeClub = URIRef(SO[idU1])
    AwayClub = URIRef(SO[idU2])
    g.add((Game, SO['homeClub'], HomeClub))
    g.add((Game, SO['awayClub'], AwayClub))    
    g.add((Game, SO['matchDay'], Literal(row['date'], datatype=XSD.date)))
    g.add((Game, SO['homeClubGoals'], Literal(row['home_club_goals'], datatype=XSD.int)))
    g.add((Game, SO['awayClubGoals'], Literal(row['away_club_goals'], datatype=XSD.int)))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'games.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: user 404 ms, sys: 3.27 ms, total: 407 ms
Wall time: 409 ms


In [ ]:
# reload the full players dataframe
players = pd.read_csv(playersUrl, sep=',', index_col='player_id', keep_default_na=False, na_values=['_'])
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("so", SO)

In [ ]:
oldgameid = ''
for index, row in apps.iterrows():
    idA = "appearance"+str(index)
    idP = "player"+str(row['player_id'])
    Appearance = URIRef(SO[idA])
    Player = URIRef(SO[idP])
    currgameid = str(row['game_id'])
    idG = "game"+currgameid
    Game = URIRef(SO[idG])
    g.add((Appearance, RDF.type, SO.Appearance))
    g.add((Player, SO['appearIn'], Appearance))
    g.add((Appearance, SO['playIn'], Game))

    g.add((Appearance, SO['goals'], Literal(row['goals'], datatype=XSD.int)))
    g.add((Appearance, SO['assists'], Literal(row['assists'], datatype=XSD.int)))
    g.add((Appearance, SO['minutesPlayed'], Literal(row['minutes_played'], datatype=XSD.int)))
    g.add((Appearance, SO['yellowCard'], Literal(row['yellow_cards'], datatype=XSD.int)))
    g.add((Appearance, SO['redCard'], Literal(row['red_cards'], datatype=XSD.int)))

    #add this triple only once per game
    if (currgameid != oldgameid):
        idL = "league"+str(row['league_id'])
        g.add((Game, SO['belongTo'], URIRef(SO[idL])))
        oldgameid = currgameid

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'appearances.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: user 12.2 s, sys: 48.5 ms, total: 12.3 s
Wall time: 12.3 s
